
Step 1. [Download history price and clean data](/notebooks/forex/hawkeye/1_download.ipynb)

Step 2. [Use monte carlo simulate each stock's movement for 10000 times](/notebooks/forex/hawkeye/2_process.ipynb)

**Step 3. Apply some heuristic rules to give stock recommendation**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('/opt/hawkeye')
from asx import *

path = '/opt/hawkeye/data/20190531/result.csv'
last_path = '/opt/hawkeye/data/20190524/result.csv'

df = pd.read_csv(path,
                 usecols=[
                     'code', 'last_date', 'start price', 'sim_mean',
                     'sim_diff', 'VaR 99%', 'VaR 99% Percent', 'volume_mean',
                     'return_mean', 'return_sigma'
                 ],
                 index_col='code')
last_df = pd.read_csv(last_path,
                      usecols=[
                          'code', 'last_date', 'start price', 'sim_mean',
                          'sim_diff', 'VaR 99%', 'VaR 99% Percent',
                          'volume_mean', 'return_mean', 'return_sigma'
                      ],
                      index_col='code')
df.head()
df['return'] = round(df['sim_diff'] / df['start price'] * 100, 3)
df['last_return'] = round(last_df['sim_diff'] / last_df['start price'] * 100,
                          3)
df['return_increase'] = df['return'] - df['last_return']
df['week_return'] = round(
    (df['start price'] - last_df['start price']) / df['start price'] * 100, 3)
df.drop(columns=['sim_diff', 'sim_mean', 'VaR 99%'], inplace=True)

print(f'Total stock number = {len(df)}')

print(df['start price'].quantile(0.7))
print('Sim return 90%% percentile = %s' % df['return'].quantile(0.9))

volume_threshold = df['volume_mean'].quantile(0.1)
print(f'Volume 90% percentile = {volume_threshold}')

df = df[(df['volume_mean'] > volume_threshold) & (df['start price'] > 0.1)]
print(len(df))
df.sort_values(by='return', ascending=False).head(20)

Total stock number = 1356
1.0224999999999993
Sim return 90% percentile = 1.2295
Volume 90% percentile = 33482.06060606061
724


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
FTC,2019-05-31,0.155,32.9094,8.031491e+05,0.024874,0.190538,5.325,NaN,NaN,NaN
ANO,2019-05-31,6.760,10.8810,1.014060e+05,0.021026,0.066211,4.132,4.705,-0.573,-3.846
TNY,2019-05-31,1.275,19.1310,1.069871e+05,0.019912,0.106839,4.031,NaN,NaN,NaN
FEX,2019-05-31,0.105,15.9794,8.738712e+06,0.020393,0.091461,3.991,3.483,0.508,10.476
AVH,2019-05-31,0.450,12.8412,8.818815e+06,0.019414,0.075731,3.937,3.976,-0.039,-1.111
8CO,2019-05-31,0.155,13.0775,2.571469e+05,0.017824,0.076189,3.486,3.636,-0.150,0.000
LTR,2019-05-31,0.105,16.9439,1.225242e+07,0.018377,0.094203,3.442,3.816,-0.374,-4.762
WZR,2019-05-31,0.150,17.7767,5.728785e+06,0.017412,0.100354,3.375,3.627,-0.252,-13.333
ISX,2019-05-31,0.620,9.1959,2.139066e+06,0.015591,0.056436,3.123,3.180,-0.057,-6.452


In [43]:
# high mean return 
df.sort_values(by='return', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
FTC,2019-05-31,0.155,32.9094,8.031491e+05,0.024874,0.190538,5.325,NaN,NaN,NaN
ANO,2019-05-31,6.760,10.8810,1.014060e+05,0.021026,0.066211,4.132,4.705,-0.573,-3.846
TNY,2019-05-31,1.275,19.1310,1.069871e+05,0.019912,0.106839,4.031,NaN,NaN,NaN
FEX,2019-05-31,0.105,15.9794,8.738712e+06,0.020393,0.091461,3.991,3.483,0.508,10.476
AVH,2019-05-31,0.450,12.8412,8.818815e+06,0.019414,0.075731,3.937,3.976,-0.039,-1.111
8CO,2019-05-31,0.155,13.0775,2.571469e+05,0.017824,0.076189,3.486,3.636,-0.150,0.000
LTR,2019-05-31,0.105,16.9439,1.225242e+07,0.018377,0.094203,3.442,3.816,-0.374,-4.762
WZR,2019-05-31,0.150,17.7767,5.728785e+06,0.017412,0.100354,3.375,3.627,-0.252,-13.333
ISX,2019-05-31,0.620,9.1959,2.139066e+06,0.015591,0.056436,3.123,3.180,-0.057,-6.452


In [44]:
# high price
df[df['start price']>1].sort_values(by='return', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
ANO,2019-05-31,6.760,10.8810,1.014060e+05,0.021026,0.066211,4.132,4.705,-0.573,-3.846
TNY,2019-05-31,1.275,19.1310,1.069871e+05,0.019912,0.106839,4.031,NaN,NaN,NaN
DUB,2019-05-31,1.240,10.2016,6.239950e+05,0.013108,0.057602,2.720,2.199,0.521,19.758
Z1P,2019-05-31,3.200,7.2795,2.529376e+06,0.012170,0.043293,2.473,2.412,0.061,-8.125
BUB,2019-05-31,1.245,9.6117,5.404982e+06,0.011787,0.053695,2.314,2.362,-0.048,-1.606
CIA,2019-05-31,3.180,6.5923,2.994197e+05,0.010987,0.039603,2.150,1.979,0.171,8.805
5GN,2019-05-31,1.335,8.3356,3.498933e+05,0.010913,0.046615,2.119,2.579,-0.460,-1.124
JIN,2019-05-31,18.000,4.3880,3.210655e+05,0.009599,0.027849,1.867,1.802,0.065,4.833
CRD,2019-05-31,1.690,8.0863,2.845716e+05,0.009651,0.044229,1.849,1.750,0.099,1.775


In [45]:
# highest price
df[df['return']>0.8].sort_values(by='start price', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
MFG,2019-05-31,43.43,2.8858,5.610274e+05,0.006196,0.018218,1.195,1.330,-0.135,-1.082
CUV,2019-05-31,34.49,6.3325,1.288410e+05,0.006577,0.033461,1.301,1.339,-0.038,2.349
APX,2019-05-31,27.88,5.6295,8.151082e+05,0.007906,0.031844,1.524,1.559,-0.035,5.882
APT,2019-05-31,24.04,7.2215,2.064414e+06,0.006804,0.038466,1.276,1.505,-0.229,0.458
JIN,2019-05-31,18.00,4.3880,3.210655e+05,0.009599,0.027849,1.867,1.802,0.065,4.833
IEL,2019-05-31,17.09,4.6516,7.190197e+05,0.005415,0.025465,1.017,1.111,-0.094,2.926
BRG,2019-05-31,17.05,4.4322,3.105192e+05,0.004989,0.023380,0.953,1.061,-0.108,-1.349
IVC,2019-05-31,15.63,3.5790,5.106814e+05,0.004193,0.020111,0.814,0.948,-0.134,-2.175
LOV,2019-05-31,11.40,5.8615,3.129449e+05,0.006490,0.031582,1.244,1.305,-0.061,-1.754


In [46]:
# low risk
var_threshold = df['VaR 99% Percent'].quantile(0.3)
print(var_threshold)
df[df['VaR 99% Percent'] < var_threshold].sort_values(by='return',
                                                      ascending=False).head(10)

4.30028


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
AD8,2019-05-31,7.70,4.0039,1.916764e+05,0.007712,0.024081,1.531,1.440,0.091,7.532
DDR,2019-05-31,5.14,3.4923,1.524193e+05,0.006631,0.021177,1.297,1.264,0.033,0.389
APE,2019-05-31,10.45,3.7749,5.357645e+04,0.006103,0.022147,1.222,0.984,0.238,9.091
MFG,2019-05-31,43.43,2.8858,5.610274e+05,0.006196,0.018218,1.195,1.330,-0.135,-1.082
IVC,2019-05-31,15.63,3.5790,5.106814e+05,0.004193,0.020111,0.814,0.948,-0.134,-2.175
ASB,2019-05-31,2.91,3.4316,9.751107e+05,0.004273,0.018414,0.801,0.841,-0.040,1.718
SSM,2019-05-31,2.48,3.3933,1.465714e+06,0.003921,0.018382,0.758,0.766,-0.008,-0.605
NCK,2019-05-31,7.07,3.5470,1.991618e+05,0.003948,0.019250,0.747,0.739,0.008,2.263
AMA,2019-05-31,1.26,3.7134,8.944539e+05,0.003808,0.019113,0.736,0.770,-0.034,0.397


In [47]:
# high volume
df.sort_values(by='volume_mean', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
TLS,2019-05-31,3.650,2.2942,2.722267e+07,0.002560,0.012251,0.495,0.476,0.019,3.014
FMG,2019-05-31,8.050,4.9030,1.978598e+07,0.007215,0.027886,1.377,1.673,-0.296,-4.969
S32,2019-05-31,3.320,3.3291,1.878719e+07,0.000083,0.014601,-0.002,0.215,-0.217,-4.669
HSO,2019-05-31,2.460,1.3034,1.378691e+07,0.001068,0.006775,0.202,0.253,-0.051,0.000
MGR,2019-05-31,3.030,1.9111,1.300801e+07,0.003205,0.010838,0.601,0.690,-0.089,-2.970
VCX,2019-05-31,2.580,2.3597,1.239887e+07,0.000134,0.010585,0.027,0.156,-0.129,-4.651
AWC,2019-05-31,2.370,3.5380,1.226958e+07,0.001512,0.017210,0.307,0.364,-0.057,-2.532
SCG,2019-05-31,3.800,2.4471,1.225611e+07,-0.000006,0.010798,0.001,0.084,-0.083,-3.158
LTR,2019-05-31,0.105,16.9439,1.225242e+07,0.018377,0.094203,3.442,3.816,-0.374,-4.762


In [48]:
# high return increase
df.sort_values(by='return_increase', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
CPV,2019-05-31,0.280,11.3910,2.124105e+05,0.003062,0.055347,0.732,-0.059,0.791,16.071
KNO,2019-05-31,0.105,11.3152,1.818797e+05,0.007222,0.057007,1.525,0.791,0.734,23.810
COB,2019-05-31,0.170,11.8985,2.644598e+05,-0.000125,0.055141,0.036,-0.574,0.610,20.588
TDO,2019-05-31,0.115,11.9863,1.090814e+05,0.004257,0.059747,0.859,0.267,0.592,20.870
ALC,2019-05-31,0.115,10.4934,2.633581e+06,0.011006,0.056086,2.139,1.568,0.571,20.000
LYC,2019-05-31,3.050,9.4248,6.111242e+06,0.007458,0.049616,1.564,1.019,0.545,21.967
CVN,2019-05-31,0.540,7.3545,3.551803e+06,0.005822,0.038221,1.127,0.598,0.529,21.296
HRL,2019-05-31,0.125,7.6885,2.877578e+05,0.003922,0.038397,0.774,0.249,0.525,16.000
DUB,2019-05-31,1.240,10.2016,6.239950e+05,0.013108,0.057602,2.720,2.199,0.521,19.758


In [6]:
# highest market cap.
highest_market_cap = [
    'BHP', 'RIO', 'CBA', 'WBC', 'CSL', 'ANZ', 'NAB', 'MQG', 'WOW', 'WES',
    'TLS', 'WPL'
    'S32', 'FMG', 'BXB', 'NCM', 'IAG', 'SUN', 'AMC', 'QBE', 'CIM', 'ALL',
    'AGL', 'ASX', 'RHC', 'OSH', 'SHL', 'TWE', 'REA', 'COH'
]
asx_20 = get_asx_20_list()
df[df.index.isin(asx_20)].sort_values(by='return', ascending=False).head(20)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
RIO,2019-05-31,100.30,2.6950,1.820674e+06,0.002742,0.014172,0.553,0.648,-0.095,-2.712
TLS,2019-05-31,3.65,2.2942,2.722267e+07,0.002560,0.012251,0.495,0.476,0.019,3.014
AMC,2019-05-31,15.98,1.2910,3.699257e+06,0.002279,0.007633,0.438,0.454,-0.016,-1.126
BXB,2019-05-31,12.16,1.8228,3.755811e+06,0.001909,0.009773,0.370,0.444,-0.074,-0.247
TCL,2019-05-31,13.93,1.3863,5.211636e+06,0.001907,0.007810,0.369,NaN,NaN,NaN
WES,2019-05-31,37.05,2.4519,2.546650e+06,0.001779,0.012437,0.343,0.430,-0.087,-2.969
ANZ,2019-05-31,28.01,2.9527,5.637911e+06,0.001605,0.014416,0.308,0.394,-0.086,0.678
IAG,2019-05-31,7.65,2.2761,5.398345e+06,0.001322,0.010963,0.272,0.301,-0.029,-3.137
WPL,2019-05-31,35.42,2.2891,2.413565e+06,0.001310,0.011268,0.258,0.350,-0.092,-1.652


In [60]:
# high last week return
df.sort_values(by='week_return', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
KNO,2019-05-31,0.105,11.3152,1.818797e+05,0.007222,0.057007,1.525,0.791,0.734,23.810
GGG,2019-05-31,0.130,12.1325,1.825979e+06,0.008462,0.061919,1.643,1.170,0.473,23.077
LYC,2019-05-31,3.050,9.4248,6.111242e+06,0.007458,0.049616,1.564,1.019,0.545,21.967
CVN,2019-05-31,0.540,7.3545,3.551803e+06,0.005822,0.038221,1.127,0.598,0.529,21.296
SFX,2019-05-31,0.475,8.1576,1.844433e+05,-0.002101,0.034691,-0.426,-0.900,0.474,21.053
TDO,2019-05-31,0.115,11.9863,1.090814e+05,0.004257,0.059747,0.859,0.267,0.592,20.870
COB,2019-05-31,0.170,11.8985,2.644598e+05,-0.000125,0.055141,0.036,-0.574,0.610,20.588
ALC,2019-05-31,0.115,10.4934,2.633581e+06,0.011006,0.056086,2.139,1.568,0.571,20.000
DUB,2019-05-31,1.240,10.2016,6.239950e+05,0.013108,0.057602,2.720,2.199,0.521,19.758


## Thisk week recommendations

In [54]:
# recommendations
selection = [
    'ANO',  # high return & high risk
    'ISX',  # high return & high risk
    'WKT',  # high return & low price & media risk
    'Z1P',  # high return & media risk
    'AD8',  # low risk & high price
    'LTR',  # high return & high volume
    'DUB',  # high increase
    'ALC',  # high increase
]
df[df.index.isin(selection)].sort_values(by='return', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
ANO,2019-05-31,6.760,10.8810,1.014060e+05,0.021026,0.066211,4.132,4.705,-0.573,-3.846
LTR,2019-05-31,0.105,16.9439,1.225242e+07,0.018377,0.094203,3.442,3.816,-0.374,-4.762
ISX,2019-05-31,0.620,9.1959,2.139066e+06,0.015591,0.056436,3.123,3.180,-0.057,-6.452
DUB,2019-05-31,1.240,10.2016,6.239950e+05,0.013108,0.057602,2.720,2.199,0.521,19.758
WKT,2019-05-31,0.355,8.7626,7.694484e+05,0.013725,0.052205,2.672,2.707,-0.035,11.268
Z1P,2019-05-31,3.200,7.2795,2.529376e+06,0.012170,0.043293,2.473,2.412,0.061,-8.125
ALC,2019-05-31,0.115,10.4934,2.633581e+06,0.011006,0.056086,2.139,1.568,0.571,20.000
AD8,2019-05-31,7.700,4.0039,1.916764e+05,0.007712,0.024081,1.531,1.440,0.091,7.532


## Last week review

In [4]:
selection = [
    'AVH',  # high return & high risk
    'ANO',  # high return & high risk
    'AGH',  # high return & meduim risk
    'Z1P',  # medium price & risk
    'MFG',  # high price & low risk
    'AD8',  # low risk
    'FMG',  # high volume & low risk
]
df[df.index.isin(selection)].sort_values(by='return', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
ANO,2019-05-31,6.76,10.8810,1.014060e+05,0.021026,0.066211,4.132,4.705,-0.573,-3.846
AVH,2019-05-31,0.45,12.8412,8.818815e+06,0.019414,0.075731,3.937,3.976,-0.039,-1.111
Z1P,2019-05-31,3.20,7.2795,2.529376e+06,0.012170,0.043293,2.473,2.412,0.061,-8.125
AGH,2019-05-31,0.74,7.9739,4.073074e+05,0.010303,0.044561,2.012,2.032,-0.020,10.811
AD8,2019-05-31,7.70,4.0039,1.916764e+05,0.007712,0.024081,1.531,1.440,0.091,7.532
FMG,2019-05-31,8.05,4.9030,1.978598e+07,0.007215,0.027886,1.377,1.673,-0.296,-4.969
MFG,2019-05-31,43.43,2.8858,5.610274e+05,0.006196,0.018218,1.195,1.330,-0.135,-1.082


In [5]:
week_return = round(sum(df[df.index.isin(selection)]['week_return']), 3)
print('Last week return:', week_return)

Last week return: -0.79


## Signle performance review

In [2]:
df[df.index=='FCT']

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
FCT,2019-05-31,0.29,10.2824,434748.020202,0.007419,0.053854,1.446,1.466,-0.02,-10.345
